<a href="https://colab.research.google.com/github/dzivkovi/LLM_RBAC_experiments/blob/main/02_LlamaIndex_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-index
!pip install -q openai
!pip install -q transformers
!pip install -q accelerate
!pip install -q optimum[exporters]
!pip install -q InstructorEmbedding
!pip install -q sentence_transformers
!pip install -q pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

### Load Data like in Tutorial 01


In [ ]:
from llama_index.llms import OpenAI
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [ ]:
!mkdir -p 'data'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/examples/paul_graham_essay/data/paul_graham_essay.txt' -O 'data/paul_graham_essay.txt'

--2024-01-05 02:39:09--  https://raw.githubusercontent.com/run-llama/llama_index/main/examples/paul_graham_essay/data/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham_essay.txt’

data/paul_graham_es 100%[===================>]  73.28K  --.-KB/s    in 0.009s  

2024-01-05 02:39:09 (7.91 MB/s) - ‘data/paul_graham_essay.txt’ saved [75042/75042]



In [ ]:
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up? ")

In [ ]:
display(Markdown(f"<b>{response}</b>"))

<b>The author wrote short stories and worked on programming, specifically on an IBM 1401 computer in 9th grade. They also mentioned writing simple games, a program to predict rocket heights, and a word processor on a TRS-80 computer.</b>

### Customize Large Language Model

In [ ]:
from llama_index import ServiceContext, set_global_service_context


# define LLM: https://gpt-index.readthedocs.io/en/latest/core_modules/model_modules/llms/usage_custom.html
llm = OpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=256)

# configure service context
service_context = ServiceContext.from_defaults(llm=llm,
                                               chunk_size=800,
                                               chunk_overlap=20)
# set_global_service_context(service_context)
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

### Customize Embeddings Model:

In [ ]:
from llama_index import ServiceContext
from llama_index.embeddings import OpenAIEmbedding


In [ ]:
embed_model_openai = OpenAIEmbedding()
service_context = ServiceContext.from_defaults(embed_model=embed_model_openai)

In [ ]:
text_embeddings = embed_model_openai.get_text_embedding("AI is awesome!")

In [ ]:
len(text_embeddings)

1536

In [ ]:
text_embeddings[:10]

[-0.012663803063333035,
 -0.01700890064239502,
 0.0014917614171281457,
 -0.0233109500259161,
 -0.012338750064373016,
 0.013811439275741577,
 -0.010885961353778839,
 -0.00037812296068295836,
 0.012743407860398293,
 -0.04173946753144264]

### Local Embeddings with HuggingFace

Massive Text Embedding Benchmark (MTEB) [Leaderboard](https://huggingface.co/spaces/mteb/leaderboard):

In [ ]:
from llama_index.embeddings import HuggingFaceEmbedding
embed_model_bge = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
text_embeddings = embed_model_bge.get_text_embedding("AI is awesome!")
print(len(text_embeddings))

384


### InstructorEmbedding

In [ ]:
from llama_index.embeddings import InstructorEmbedding
embed_model_inst = InstructorEmbedding(model_name="hkunlp/instructor-base")

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.2k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
text_embeddings = embed_model_inst.get_text_embedding("AI is awesome!")
print(len(text_embeddings))

768


### Benchmarking


In [ ]:
!curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.7M  100 20.7M    0     0   250k      0  0:01:24  0:01:24 --:--:--  263k


In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
documents = SimpleDirectoryReader(
    input_files=["IPCC_AR6_WGII_Chapter03.pdf"]
).load_data()

In [ ]:
# Number of pages in the file, a.k.a. "documents"
print(f"Number of documents : {len(documents)}")

Number of documents : 172


### OpenAI's Embeddings



In [ ]:
embed_model_openai = OpenAIEmbedding()
service_context = ServiceContext.from_defaults(embed_model=embed_model_openai)

In [ ]:
%%timeit -r 2 -n 2
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, show_progress=True
)

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

9.81 s ± 381 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


### BGE-Small Embeddings

In [ ]:
service_context = ServiceContext.from_defaults(embed_model=embed_model_bge)

In [ ]:
%%timeit -r 2 -n 2
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, show_progress=True)

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

9.4 s ± 113 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


### Instructors Embeddings

In [ ]:
service_context = ServiceContext.from_defaults(embed_model=embed_model_inst)

In [ ]:
%%timeit -r 2 -n 2
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, show_progress=True)

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/459 [00:00<?, ?it/s]

20.4 s ± 522 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
